### 操作說明：
* 方向鍵 ↑ ↓ ← → ：移動玩家的戰機

### 遊戲機制
* 分數計算的方式：
    1. 生存獎勵：每存活1秒獲得1分
    2. 擊殺獎勵：擊殺敵人獲得分數
* 擊殺敵人的方式：
    1. 用子彈擊殺：獲得正常的分數和經驗值 (還沒寫出來QQ)
    2. 直接撞毀：獲得的分數和經驗值減半，而且玩家必須受到敵人攻擊力三倍的傷害
* 敵人會隨著時間經過越來越強 (目前敵人只有「隕石」，隨著時間經過移動速度會慢慢增加，出現頻率也會愈來愈高)
    
### 其他注意事項
* 程式還沒整理所以看起來很亂 > <

In [1]:
import pygame, sys
from pygame.locals import Color, QUIT, MOUSEBUTTONDOWN, USEREVENT, USEREVENT
from pygame.sprite import Group
from random import random, randint

score = 0
FPS = 60
player_width = 96
player_height = 96
bg_color = (0, 0, 0) # 背景顏色
screen_width = 1200
screen_height = 600

# 玩家操控的戰機的類別
class Player(pygame.sprite.Sprite):
    def __init__(self, width = player_width, height = player_height, level = 1, expRequired = 20, hpMax = 108, atk = 12):
        super().__init__()
        
        self.width = width
        self.height = height
        
        self.raw_image = pygame.image.load('./jet.png').convert_alpha()
        self.image = pygame.transform.scale(self.raw_image, (self.width, self.height))
        self.rect = self.image.get_rect()
        self.rect.topleft = (200, 300)
                             
        self.level = level # 玩家的等級
        self.expRequired = expRequired # 玩家升級所需的經驗值
        self.hpMax = hpMax # 玩家的生命上限
        self.atk = atk # 玩家的攻擊力
        self.hp = hpMax # 一開始的血量等於玩家的生命上限

        
    def moveRight(self, pixels):
        self.rect.x += pixels
 
    def moveLeft(self, pixels):
        self.rect.x -= pixels

    def moveUp(self, pixels):
        self.rect.y -= pixels
        
    def moveDown(self, pixels):
        self.rect.y += pixels

# 類別：隕石(敵人)
# AI：直線快速前進
class Enemy(pygame.sprite.Sprite):
    def __init__(self, imgDir, width = 128, height = 128, init_x = 1500, init_y = 300, hpMax = 50, 
                 atk = 15, speed = 0.5, score = 3):
        
        super().__init__()
        
        self.width = width
        self.height = width
        
        self.raw_image = pygame.image.load(imgDir).convert_alpha()
        self.image = pygame.transform.scale(self.raw_image, (width, height))
        self.rect = self.image.get_rect()
        self.rect.topleft = (init_x, randint(0 - int(self.height / 2), screen_height - int(self.height / 2))) 
            # y的範圍介於 視窗大小 + 半個自身高度 之間 (鼓勵玩家不要躲在邊邊)
        
        self.floating_point_x = init_x
        
        self.hpMax = hpMax # 玩家的生命上限
        self.atk = atk # 自己的攻擊力
        self.hp = hpMax # 一開始自己的血量等於自己的生命上限
        self.speed = speed # 自己的速度
        self.score = score # 自己被玩家擊殺時，玩家可以獲得的分數(撞毀的分數減半)
    
    def update(self):
        ### 移動相關 ###
 
        # 實現每幀移動小數pixel的方法
        self.floating_point_x -= self.speed 
        self.rect.x = int(self.floating_point_x)
 
        # 超出邊界就消失
        if self.rect.x <= (0 - self.width):
            self.kill()
        
def main():
    # 初始化
    pygame.init()
    
    # 設置timer
    clock = pygame.time.Clock()
    clock2 = pygame.time.Clock()
    
    # 建立 window 視窗畫布
    window_surface = pygame.display.set_mode((screen_width, screen_height))
    # 設置視窗標題為
    pygame.display.set_caption('新手戰機 (版本：20200303)')
    # 宣告 font 文字物件
    head_font = pygame.font.SysFont('Calibri', 36)
    # 更新畫面，等所有操作完成後一次更新（若沒更新，則元素不會出現）
    pygame.display.update()
    
    score = 0
    FPS = 60
    
    
    # 創建玩家
    player = Player()
    
    # 創建存放子彈的群組
    bullets = pygame.sprite.Group()

    
    # 創建存放敵人的群組
    enemys = pygame.sprite.Group()
    speed_new = 0.5
    
    screen=pygame.display.get_surface()
    time_counter = 0
    time_counter2 = 0
    
    lower_limit = 1000
    upper_limit = 2000
    time_threshold = randint(lower_limit, upper_limit) # 1~2秒之後出現第一個敵人
    
    game_state = 'playing'
    
    # 事件迴圈監聽事件，進行事件處理
    while True:
        
        if game_state == 'playing':

            # 玩家死亡
            if player.hp <= 0:
                player.hp = 0
                game_state = 'game_over'

            # 迭代整個事件迴圈，若有符合事件則對應處理
            for event in pygame.event.get():
                # 當使用者結束視窗，程式也結束
                if event.type == QUIT:
                    pygame.quit()
                    sys.exit()

            # 玩家和敵人的碰撞偵測
            for enemy in enemys:
                if pygame.sprite.collide_rect(player, enemy):
                    enemy.hp = 0
                    player.hp -= (enemy.atk * 3)
                    score += int(enemy.score / 2)
                    enemy.kill()

            # 每過一段時間出現新的敵人
            time_counter += clock.tick()
            if time_counter >= time_threshold:
                speed_new += random() * 0.025 # 新的敵人將擁有更快的速度(每畫格增加0~0.025)
                width_new = height_new = randint(64,160) # 新的敵人會有不同的大小
                enemys.add(Enemy(imgDir = '.\meteorite.png', width = width_new, height = height_new, 
                                 speed = speed_new, score = 10)) 
                time_counter = 0
                
                lower_limit -= 25
                if lower_limit <= 250:
                    lower_limit = 250
                    
                upper_limit -= 25
                if upper_limit <= 500:
                    upper_limit = 500

                time_threshold = randint(lower_limit, upper_limit) # 一段時間後出現下一個敵人(之後出現頻率會愈來愈高)
            
            # 持續存活分數獎勵
            time_counter2 += clock2.tick()
            if time_counter2 >= 1000:
                score += 1 # 每存活1秒加1分
                time_counter2 = 0


            # 背景著色
            window_surface.fill(bg_color)

            # 玩家戰機的四個方位移動(配合按鍵)
            keys_pressed = pygame.key.get_pressed()

            if keys_pressed[pygame.K_LEFT]:
                if player.rect.x >= 0:
                    player.moveLeft(1)

            if keys_pressed[pygame.K_RIGHT]:
                if player.rect.x <= (screen_width - player.width):
                    player.moveRight(1)

            if keys_pressed[pygame.K_UP]:
                if player.rect.y >= 0:
                    player.moveUp(1)

            if keys_pressed[pygame.K_DOWN]:
                if player.rect.y <= (screen_height - player.height):
                    player.moveDown(1)

            # 敵人畫面的更新
            enemys.draw(screen)
            enemys.update()

            # 渲染物件
            if player.hp > 0:
                window_surface.blit(player.image, player.rect)

            # 玩家資訊儀表(顯示等級、生命值、分數...等資訊)
            text_surface = head_font.render('HP: {} / {}'.format(player.hp, player.hpMax), True, (255, 255, 255))
            window_surface.blit(text_surface, (10, 10))
            text_surface2 = head_font.render('Score: {}'.format(score), True, (255, 255, 255))
            window_surface.blit(text_surface2, (10, 40))

            pygame.display.update()
        
        elif game_state == 'game_over':
            
            # 迭代整個事件迴圈，若有符合事件則對應處理
            for event in pygame.event.get():
                # 當使用者結束視窗，程式也結束
                if event.type == QUIT:
                    pygame.quit()
                    sys.exit()
                    
            window_surface.fill(bg_color)
            
            # 玩家成績儀表版
            head_font2 = pygame.font.SysFont('Calibri', 72)
            text_surface2 = head_font2.render('GAME OVER', True, (255, 255, 255))
            text_rect2 = text_surface2.get_rect(center=(screen_width/2, screen_height/2))
            window_surface.blit(text_surface2, text_rect2)
            
            text_surface = head_font.render('Yor score: {}'.format(score), True, (255, 255, 255))
            text_rect = text_surface.get_rect(center=(screen_width/2, screen_height/2 * 1.2))
            window_surface.blit(text_surface, text_rect)

            pygame.display.update()



if __name__ == '__main__':
    main()
        

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


SystemExit: 

C:\Users\kevin\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3334: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
